<a href="https://colab.research.google.com/github/joaogabrielanuto/quant-traders/blob/under-construction/detector-topos-fundos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://tradingcomdados.com/images/logotipo/logotipo-trading-com-dados.svg" width="300" align="left"/>

---
# **Screening de abertura com Python** 


#### Varrendo em segundos quais ativos atendem aos critérios pré estabelecidos
---

# 1. Bibliotecas

In [ ]:
# Manipulação de dados e série temporais
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, time
import pytz
from time import sleep

# Fonte de dados ativos
import MetaTrader5 as mt5

# Visualização dos dados
import mplfinance as mpf

# 2. Exploração do código

## 2.1. Obtenção das cotações

In [ ]:
mt5.initialize()

In [ ]:
timezone = pytz.timezone("America/Sao_Paulo")
data_inicio = datetime.now() - timedelta(days=5)
data_fim = datetime.now()

In [ ]:
tickers = ['ABEV3', 'ALPA4', 'ALSO3', 'ARZZ3', 'ASAI3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4',
            'BBSE3', 'BEEF3', 'BPAC11', 'BRAP4', 'BRFS3', 'BRKM5', 'CASH3', 'CCRO3', 'CIEL3', 'CMIG4',
            'CMIN3', 'COGN3', 'CPFE3', 'CPLE6', 'CRFB3', 'CSAN3', 'CSNA3', 'CVCB3', 'CYRE3', 'DXCO3',
            'EGIE3', 'ELET3', 'ELET6', 'EMBR3', 'ENBR3', 'ENEV3', 'ENGI11', 'EQTL3', 'EZTC3', 'FLRY3',
            'GGBR4', 'GOAU4', 'GOLL4', 'HAPV3', 'HYPE3', 'IGTI11', 'IRBR3', 'ITSA4', 'ITUB4', 'JBSS3',
            'KLBN11', 'LREN3', 'LWSA3', 'MGLU3', 'MRFG3', 'MRVE3', 'MULT3', 'NTCO3', 'PCAR3', 'PETR3',
            'PETR4', 'PETZ3', 'PRIO3', 'RADL3', 'RAIL3', 'RAIZ4', 'RDOR3', 'RENT3', 'RRRP3', 'SANB11',
            'SBSP3', 'SLCE3', 'SMTO3', 'SOMA3', 'SUZB3', 'TAEE11', 'TIMS3', 'TOTS3', 'UGPA3', 'USIM5',
            'VALE3', 'VBBR3', 'VIIA3', 'VIVT3', 'WEGE3', 'YDUQ3']

In [ ]:
ls_dados_ohlc = []
indice_ticker = 0

for i in tickers:
    dados_ohlc = mt5.copy_rates_range(i, mt5.TIMEFRAME_M5, data_inicio, data_fim)
    dados_ohlc = pd.DataFrame(dados_ohlc)
    dados_ohlc['ticker'] = str(i)
    dados_ohlc['indice_ticker'] = indice_ticker
    indice_ticker += 1
    dados_ohlc.index = pd.to_datetime(dados_ohlc['time'], unit='s')
    dados_ohlc = dados_ohlc.drop('time', axis=1)

    ls_dados_ohlc.append(dados_ohlc)

In [ ]:
ls_dados_ohlc[0]

In [ ]:
list(enumerate(tickers))

# 3. Construção das funções

**Produzindo um loop**

In [ ]:
for df in ls_dados_ohlc:
    df['mm_rapida'] = df.close.rolling(20).mean()
    df['mm_lenta'] = df.close.rolling(200).mean()

In [ ]:
ls_dados_ohlc[0]

**Função médias móveis**

In [ ]:
def medias_moveis(df, mm_rapida=20, mm_lenta=200):

    df['mm_rapida'] = df.close.rolling(mm_rapida).mean()
    df['mm_lenta'] = df.close.rolling(mm_lenta).mean()
    df = df.dropna()

    return df

In [ ]:
medias_moveis(ls_dados_ohlc[0])

**Função condições do screening**

In [ ]:
df.index[-1].date()

In [ ]:
str(df.index[-1].date())

In [ ]:
ls_dados_ohlc[0].open[-1]

In [ ]:
ls_dados_ohlc[0]['mm_rapida'][-1]

In [ ]:
def screnning_abertura(df):

    cond1 = df['mm_rapida'][-1] > df['mm_lenta'][-1]
    cond2 = df['open'][-1] > df['mm_rapida'][-1]

    return (cond1 and cond2)

In [ ]:
for df in ls_dados_ohlc:

    medias_moveis(df)
    
    if screnning_abertura(df):
        print(df.indice_ticker[-1], df.ticker[-1])

**Visualização dos ativos**

In [ ]:
i = 2

mpf.plot(ls_dados_ohlc[i], type='candle', mav=(20, 200))

In [ ]:
i = 2

mm_rapida_plot = list(ls_dados_ohlc[i][['mm_rapida']].itertuples(index=True))
mm_lenta_plot = list(ls_dados_ohlc[i][['mm_lenta']].itertuples(index=True))

df_figure = ls_dados_ohlc[i].iloc[-30:]
mm_rapida_plot = mm_rapida_plot[-30:]
mm_lenta_plot = mm_lenta_plot[-30:]

mpf.plot(df_figure, type='candle', alines=dict(alines=[mm_rapida_plot, mm_lenta_plot],colors=['b','y']))

**Incluir a data na função screening**

In [ ]:
df.index[-1].date()

In [ ]:
str(df.index[-1].date())

In [ ]:
data_hoje = str(datetime.now().date())
data_hoje

In [ ]:
def ativo_ja_abriu(df, data):

    cond1 = str(df.index[-1].date())==data

    return cond1

In [ ]:
data_ontem = str( datetime.now().date() - timedelta(days=1))
data_hoje = str(datetime.now().date())
data_hoje = '2023-05-12'

for df in ls_dados_ohlc:

    medias_moveis(df)
    
    if screnning_abertura(df, data_ontem):
        print(df.indice_ticker[0], df.ticker[0])

# 4. Screnning abertura

In [ ]:
mt5.initialize()

timezone = pytz.timezone("America/Sao_Paulo")
data_inicio = datetime.now() - timedelta(days=1)
data_fim = datetime.now()

tickers = ['ABEV3', 'ALPA4', 'ALSO3', 'ARZZ3', 'ASAI3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4',
            'BBSE3', 'BEEF3', 'BPAC11', 'BRAP4', 'BRFS3', 'BRKM5', 'CASH3', 'CCRO3', 'CIEL3', 'CMIG4',
            'CMIN3', 'COGN3', 'CPFE3', 'CPLE6', 'CRFB3', 'CSAN3', 'CSNA3', 'CVCB3', 'CYRE3', 'DXCO3',
            'EGIE3', 'ELET3', 'ELET6', 'EMBR3', 'ENBR3', 'ENEV3', 'ENGI11', 'EQTL3', 'EZTC3', 'FLRY3',
            'GGBR4', 'GOAU4', 'GOLL4', 'HAPV3', 'HYPE3', 'IGTI11', 'IRBR3', 'ITSA4', 'ITUB4', 'JBSS3',
            'KLBN11', 'LREN3', 'LWSA3', 'MGLU3', 'MRFG3', 'MRVE3', 'MULT3', 'NTCO3', 'PCAR3', 'PETR3',
            'PETR4', 'PETZ3', 'PRIO3', 'RADL3', 'RAIL3', 'RAIZ4', 'RDOR3', 'RENT3', 'RRRP3', 'SANB11',
            'SBSP3', 'SLCE3', 'SMTO3', 'SOMA3', 'SUZB3', 'TAEE11', 'TIMS3', 'TOTS3', 'UGPA3', 'USIM5',
            'VALE3', 'VBBR3', 'VIIA3', 'VIVT3', 'WEGE3', 'YDUQ3']

In [ ]:
data_hoje = str(datetime.now())

while datetime.now().time() < time(8, 18, 0):
    sleep(1)


ls_dados_ohlc = []
ls_ativos_abriram = []
indice_ticker = 0

while len(ls_dados_ohlc) < len(tickers):

    for i in tickers:
        dados_ohlc = mt5.copy_rates_range(i, mt5.TIMEFRAME_M5, data_inicio, data_fim)
        dados_ohlc = pd.DataFrame(dados_ohlc)

        if ativo_ja_abriu(df, data_ontem):
            dados_ohlc['ticker'] = str(i)
            dados_ohlc['indice_ticker'] = indice_ticker
            indice_ticker += 1
            dados_ohlc.index = pd.to_datetime(dados_ohlc['time'], unit='s')
            dados_ohlc = dados_ohlc.drop('time', axis=1)
            
            ls_ativos_abriram.append(i)

            medias_moveis(dados_ohlc)

            if screnning_abertura(dados_ohlc, data_ontem):
                print(dados_ohlc.indice_ticker[0], dados_ohlc.ticker[0])

            ls_dados_ohlc.append(dados_ohlc)

In [ ]:
ls_dados_ohlc[3]

In [ ]:
i = 0

mm_rapida_plot = list(ls_dados_ohlc[i][['mm_rapida']].itertuples(index=True))
mm_lenta_plot = list(ls_dados_ohlc[i][['mm_lenta']].itertuples(index=True))

df_figure = ls_dados_ohlc[i].iloc[-50:]
mm_rapida_plot = mm_rapida_plot[-50:]
mm_lenta_plot = mm_lenta_plot[-50:]

mpf.plot(df_figure, type='candle', alines=dict(alines=[mm_rapida_plot, mm_lenta_plot],colors=['b','y']))

In [ ]:
ls_dados_ohlc = []

while datetime.now().time() < time(12, 15, 0):
    sleep(1)

while len(ls_dados_ohlc) < len(tickers):
    
    indice_ticker = 0

    for i in tickers:
        dados_ohlc = mt5.copy_rates_range(i, mt5.TIMEFRAME_M5, data_inicio, data_fim)
        dados_ohlc = pd.DataFrame(dados_ohlc)
        dados_ohlc['ticker'] = str(i)
        dados_ohlc['indice_ticker'] = indice_ticker
        indice_ticker += 1
        dados_ohlc.index = pd.to_datetime(dados_ohlc['time'], unit='s')
        dados_ohlc = dados_ohlc.drop('time', axis=1)

        medias_moveis(dados_ohlc)

        if screnning_abertura(dados_ohlc):
            print(dados_ohlc.indice_ticker[0], dados_ohlc.ticker[0])

        ls_dados_ohlc.append(dados_ohlc)